In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
import cv2
import matplotlib.pyplot as plt
import random



In [2]:
# Setup working directory
emotionList = ('Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt')

def Setup():
  os.chdir("C:\\Users\\jazzt\\anaconda3\\envs\\tflite-facial-expression\\asset") # change working directory

# currently limited to 19999 images
# Load the images into a nested list containing info of image in tuple format (label, image name)
def LoadAllImageNames(imageFiles):
  trainingSet = []
  loadCounter = 0
  for file in imageFiles:
    loadCounter += 1
    if (loadCounter%10000==0):
      print("Files loaded:{}".format(loadCounter))
      if (loadCounter > 10000):
        break
    name = file.name[:-4] # file name w/o file extension
    data = np.load("train_set\\annotations\\{}_exp.npy".format(name))
    trainingSet.append((int(data.item(0)), file.name))

  random.shuffle(trainingSet)

  return trainingSet

# Load the pixels of a picture to numpy.ndarray format. false for test set, true for training set
# Return image in RGB format
def LoadImage(imageName, trainingSetBool, normalize = True):
  intermediatePath = str()
  if (trainingSetBool):
    intermediatePath = "\\train_set\\images\\"
  else:
    intermediatePath = "\\val_set\\images\\"
  if normalize:
    image_array = cv2.imread("{}{}{}".format(os.getcwd(), intermediatePath, imageName))
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
    image_array = image_array/255
  else:
    image_array = cv2.imread("{}{}{}".format(os.getcwd(), intermediatePath, imageName))
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
  return image_array

# Extract the daata from 0 to amount from list and return it
def CropData(list, amount):
  if (len(list) < amount):
    amount = len(list)
  croppedList = list[:amount]
  del list[:amount]
  return croppedList

def LoadImages(list, trainingSetBool):
  label = []
  data = []
  for entries in list:
    try:
      image = LoadImage(entries[1], trainingSetBool)
      data.append(image)
      label.append(entries[0])
    except:
      print("Failed to load training image: ", entries[1])
  npLabel = np.array(label)
  npData = np.array(data)
  return npLabel, npData

def InitializeModel():
  pretrained_model = tf.keras.applications.MobileNetV3Large(input_shape=(224,224,3)) # Initializing model with mobile net V3 pretrained model

  # Initializing the input and output from the model, removing last layer
  base_input = pretrained_model.layers[0].input
  base_output = pretrained_model.layers[-2].output

  # Adding 3 more layers to output side
  final_output = layers.Dense(128)(base_output) # Adding new layers, to the output side
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(64)(final_output)
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(8, activation = 'softmax')(final_output) # 8 cuz there are 8 image classifications

  new_model = keras.Model(inputs = base_input, outputs = final_output)
  return new_model

def ConvertToGray(image):
  image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
  return image

def ScaleImage(image, width):
  ratio = image.shape[1]/width
  image = cv2.resize(image, (width, int(image.shape[0]/ratio)))
  return image

def DetectFace(image):
  face_roi = np.ndarray(1)
  faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
  grayImage = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  faces = faceCascade.detectMultiScale(grayImage, 1.3, 5)
  for x,y,w,h in faces:
    roi_gray = grayImage[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]
    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 2)
    facess = faceCascade.detectMultiScale(roi_gray)
    if (len(facess) == 0):
      print("Face not detected")
    else:
      for (ex,ey,ew,eh) in facess:
        face_roi = roi_color[ey:ey+eh, ex:ex+ew]
  return face_roi

def ConvertToInput(image):
  input = ScaleImage(image, 224)
  input = np.expand_dims(input, axis = 0) ## to add fourth dimension to fit model input
  input = input/255
  return input

def GetResult(model, input):
  Predictions = model.predict(input)
  Predictions[0][0] = 0
  print(Predictions)
  result = np.argmax(Predictions)
  return emotionList[result]

In [ ]:
# Get currect directory (os.getcwd() -> C:\Users\jazzt\src)

#-----------------------Start of code---------------------------
# Path directories
Setup()

# initialise image names and label
imageFiles = os.scandir("train_set\\images")
trainingSetData = LoadAllImageNames(imageFiles)

# initialise model
model = InitializeModel()
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

# train model
count = 0
while(len(trainingSetData) != 0):
  # training data
  # try:
  croppedList = CropData(trainingSetData, 100)
  print("loading image")
  label, data = LoadImages(croppedList, True)
  model.fit(data, label, epochs = 13, batch_size = 32)
  # except:
  #   print("Failed to train data")

  #clearing memory
  croppedList.clear()
  label.delete()
  data.delete()
  count = count + 1
  print("trained image count: ", count)


In [ ]:
croppedList = CropData(trainingSetData, 1000)
label, data = LoadImages(croppedList, True)

In [3]:
model = InitializeModel()
print(model.summary())
# model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  432         ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
model.fit(data, label, epochs = 13, batch_size = 4)

In [ ]:
# Testing trained model
image = LoadImage("830.jpg", False, normalize = False)
# image = cv2.resize(image, (224,224))
image = ScaleImage(image, 224)

# face = DetectFace(image)

plt.imshow(image)

preppedInput = ConvertToInput(image)
result = GetResult(new_model, preppedInput)


In [ ]:
result

In [ ]:
croppedList.clear()

In [ ]:
from keras.applications.mobilenet_v3 import MobileNetV3Large

In [ ]:
new_model = InitializeModel()
new_model.load_weights('feModelWeights2feb.h5')

In [ ]:
tf.saved_model.save(new_model, "")

In [ ]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)